<img src="logoFIUBA.jpg" width="300" align="right">


# LLMs e IAG
## TP N°2 "RAG con agentes"

Valentín Pertierra

---

## Carga de datos en base de datos vectorial

Se utilizara la base de datos vectorial de Pinecone

### Procesamiento de documentos

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# Genero los chunks
def chunkData(docs, chunk_size=100, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(docs)
    return chunks

Se cargaran en la base de datos vectorial dos CVs, el de mio y el de Juan Perez (dummy cv)

In [4]:
# Cargo los documentos 
valentinFilePath = "cv_valentin_pertierra.pdf"
juanFilePath = "cv_juan_perez.pdf"

vloader = PyPDFLoader(valentinFilePath)
vdocs = vloader.load()

jloader = PyPDFLoader(juanFilePath)
jdocs = jloader.load()

# Genero los chunks 
vchunks = chunkData(vdocs, chunk_size=500, chunk_overlap=100)
jchunks = chunkData(jdocs, chunk_size=500, chunk_overlap=100)

### Generación de embbeding y carga en base de datos vectorial
Se utilizara un indice por cada uno de los cv

In [8]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv

In [9]:
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")


In [10]:
#Connect to DB Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)

cloud = 'aws'
region = 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

indices = ['vagent', 'jagent']
namespace = "espacio"
dimension = 384

In [11]:
pc.list_indexes().names()

['raglse']

In [12]:
# Elimino el indice si es que ya existe en la base de datos
for index_name in indices:
    if index_name in pc.list_indexes().names():
      pc.delete_index(index_name)
      print("index {} borrado".format(index_name))
    
    if index_name not in pc.list_indexes().names():
        # Como lo borre en el paso anterior siempre deberia entrar aca
        print("index creado con el nombre: {}".format(index_name))
        pc.create_index(
            index_name,
            dimension=dimension, 
            metric='cosine',
            spec=spec
            )
    else:
        print("el index con el nombre {} ya estaba creado".format(index_name))
    

index creado con el nombre: vagent
index creado con el nombre: jagent


In [13]:
# Cargo un modelo de embeddings compatible
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\DELL\AppData\Local\Temp\ipykernel_10984\1365815625.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [14]:
# Cargo chunks en base de datos
docsearch = PineconeVectorStore.from_documents(
    documents=vchunks,
    index_name='vagent',
    embedding=embedding_model, 
    namespace=namespace
)
print("upserted values to 'vagent' index")

time.sleep(1)

upserted values to jagent index


In [15]:
# Cargo chunks en base de datos
docsearch = PineconeVectorStore.from_documents(
    documents=jchunks,
    index_name='jagent',
    embedding=embedding_model, 
    namespace=namespace
)
print("upserted values to 'jagent' index")

time.sleep(1)

upserted values to 'jagent' index


### Busquedas en base de datos

Realizo pruebas para verificar que los datos se guardaron correctamente

#### Valentin Pertierra

In [16]:
index = pc.Index(indices[0])
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'espacio': {'vector_count': 10}},
 'total_vector_count': 10}

In [17]:
vectorstore = PineconeVectorStore(
    index_name=indices[0],
    embedding=embedding_model,
    namespace=namespace,
)
retriever=vectorstore.as_retriever()

In [19]:
query = "Cuales es su formacion academica?"
vectorstore.similarity_search(query, k=3)

[Document(id='c2b36f28-7cf7-45db-9cdc-bea1d0fecf31', metadata={'page': 0.0, 'source': 'cv_valentin_pertierra.pdf'}, page_content='PERFIL    _________________________________________  \n \nIngeniero en electrónica con experiencia en el desarrollo de \naplicaciones para la industria 4.0. Considero que tengo constancia y \nfirmeza para alcanzar los objetivos que me propongo, ideando \nsoluciones a las dificultades que puedan surgir. Me gusta afrontar \nnuevos desafíos, buscando siempre aprender y mejorar, tanto en lo \nhumano, como en lo académico y profesional.  \n \n \n \n \n \nEXPERENCIA LABORAL \n \nAnalista de Transformación Digital'),
 Document(id='23eacb6b-943b-4022-98f5-06b901ed9546', metadata={'page': 0.0, 'source': 'cv_valentin_pertierra.pdf'}, page_content='Cursado hasta el nivel 7 \n \nE.T.N°28 “República Francesa”  \nPromedio 9,34 \n \n___________________________________________________________ \n \nExamen First Certificate in English  \nNivel B2 (MCER) acreditado por Univers

#### Juan Perez

In [20]:
index = pc.Index(indices[0])
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'espacio': {'vector_count': 10}},
 'total_vector_count': 10}

In [21]:
jvectorstore = PineconeVectorStore(
    index_name=indices[1],
    embedding=embedding_model,
    namespace=namespace,
)
retriever=jvectorstore.as_retriever()

In [23]:
query = "Cuales es su educacion o formacion academica?"
jvectorstore.similarity_search(query, k=3)

[Document(id='569c9091-edb5-4f33-8153-2a063ec08a9d', metadata={'page': 2.0, 'source': 'cv_juan_perez.pdf'}, page_content='• Aprendí sobre buenas prácticas de desarrollo en entornos empresariales. \n \nEducación \nMaestría en Inteligencia Artificial y Aprendizaje Automático \nUniversidad de la Innovación \n2015 - 2017 \nEspecialización en Ciencia de Datos \nInstituto Tecnológico Nacional \n2013 - 2014 \nLicenciatura en Ingeniería en Sistemas \nUniversidad Politécnica de Tecnología \n2007 - 2011 \nDiplomado en Ciberseguridad \nAcademia de Tecnología Avanzada \n2010 \n \nProyectos Destacados \n1. Plataforma de Predicción de Demanda:'),
 Document(id='c2633fb1-afb2-49ef-8eaa-3f0462f88871', metadata={'page': 0.0, 'source': 'cv_juan_perez.pdf'}, page_content='Información Personal \nNombre Completo: Juan Pérez Rodríguez \nDirección: Calle Falsa 123, Ciudad, País \nTeléfono: +1 234 567 890 \nCorreo Electrónico: juan.perez@example.com \nLinkedIn: linkedin.com/in/juan-perez \nGitHub: github.com/j

## Agente LLM

In [31]:
%pip install langgraph

     ------------------------------------ 126.8/126.8 KB 746.9 kB/s eta 0:00:00
     -------------------------------------- 74.7/74.7 KB 693.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\DELL\Documents\LSE\LLM e IA Gen\tps\TP1\ragenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [46]:
%pip install pygraphviz

     -------------------------------------- 106.0/106.0 KB 6.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [48 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-310\pygraphviz
  copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-310\pygraphviz
  copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-310\pygraphviz
  copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-310\pygraphviz
  copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-310\pygraphviz
  copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-310\pygraphviz
  creating build\lib.win-amd64-cpython-310\pygraphviz\tests
  copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-310\pygraphviz\tests
  copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-cpython-310\pygraphviz\tests
  copying pygraphviz\tests\test_close.py -> build\lib.win-amd64-cpython-310\pygraphviz\tests
  copying pygraphviz\tests\test_drawing.py -> build\lib.win-amd6

In [34]:
import os
from groq import Groq

from langchain.chains import ConversationChain, LLMChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph, END
from langchain import hub
from langchain.prompts import PromptTemplate

In [25]:
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [38]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY, 
    model_name='llama3-8b-8192'
)

In [39]:
# Defino una clase para guardar el estado 
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    individual: str
    history: List[str] 

# Defino un tamplate para el prompt
prompt = PromptTemplate(
    input_variables=["context", "question", "individual"],
    template="""
Eres un asistente para tareas de preguntas y respuestas. Usa los siguientes fragmentos de historia y contexto recuperados para responder la pregunta respecto al individuo.
Si no sabes la respuesta, di que no lo sabes. Usa un máximo de 200 palabras y mantén la respuesta concisa. 
---
Historia:
{history}
---
Contexto:
{context}
---
Individuo:
{individual}
---
Pregunta: {question}
Respuesta:
"""
)


# Defino una clase agente para hacer la busqueda en la base vectorial segun la persona
class Agent:
    
    def __init__(self, embedding_model, index=""):
        if index=="":
            raise ValueError("No se especifico un índice válido.")
        
        self.index = index
        self.embedding_model = embedding_model

        self.vectorstore = PineconeVectorStore(
            index_name=index,
            embedding=self.embedding_model,
            namespace=namespace,
        )

    def get_context(self,state: State):
        retrieved_docs = self.vectorstore.similarity_search(state["question"],k=2)
        return {"context": retrieved_docs}

In [40]:
# Instancio agentes
vagent = Agent(embedding_model,"vagent")
jagent = Agent(embedding_model,"jagent")

In [41]:
# Defino los nodos para el agente
def generate(state: State):
    if state["context"]:
        docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    else: 
        docs_content = ""
    # Formateo la historia como un unico string
    history = "\n".join(state["history"])
    
    # Invoco el prompt con contexto e historia previa
    messages = prompt.invoke({
        "question": state["question"],
        "context": docs_content,
        "individual": state["individual"],
        "history": history
    })

    # print(messages)
    response = llm.invoke(messages)
    
    state["history"].append(f"Q: {state['question']} A: {response.content}")
    
    # Ahora ya es posible devolver la respuesta
    return {"answer": response.content}

# Nodo para limpiar el contexto
def empty_context(state:State):
    return {"context":[]}

# Segun sobre a quien se refiere la pregunta se utiliza un agente u otro
def decide(state: State):
    
    leandro_pattern = r"(Leandro\sSaraco|Leandro|Saraco)"
    elon_pattern = r"(Elon\sMusk|Elon|Musk)"
    individual = "" #Default
    if re.search(leandro_pattern, state["question"], re.IGNORECASE):
        individual = "leandro"
    elif re.search(elon_pattern, state["question"], re.IGNORECASE):
        individual = "elon"
    return {"individual":individual}

# Funcion para determinar cuál es el próximo nodo
def decision_read_state(state:State):
    """Obtiene el individuo desde el state y lo retorna para decidir por qué nodo continuar."""
    indiv = state["individual"]
    if indiv=="":
        print("La pregunta no habla de ningun individuo.")
        return "no_individual"
    print("La pregunta habla sobre el individuo:",indiv)
    return indiv

In [44]:
# Armo el grafo de nodos
graph_builder = StateGraph(State)
graph_builder.add_node("decision",decide)
graph_builder.add_node("empty_context",empty_context)
graph_builder.add_node("valentin_context",vagent.get_context)
graph_builder.add_node("juan_context",jagent.get_context)
graph_builder.add_node("generate",generate)
graph_builder.add_conditional_edges(
    "decision",
    decision_read_state,
    {"leandro": "valentin_context","elon": "juan_context","no_individual":"empty_context"}
    )
graph_builder.add_edge("valentin_context","generate")
graph_builder.add_edge("juan_context","generate")
graph_builder.add_edge("empty_context","generate")
graph_builder.set_entry_point("decision")
graph = graph_builder.compile()

In [45]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.